In [1]:
import numpy as np
import pandas as pd

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import imshow

from rasterio.merge import merge
import geopandas as gdp
from rasterio.mask import mask
from rasterio.plot import adjust_band
from rasterio.plot import reshape_as_raster, reshape_as_image
from shapely.geometry import mapping
import sys
from tqdm import tqdm
from PIL import Image
from PIL.ImageOps import expand
import os

import gc
import psutil
import shutil

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
### Read shapefiles
train = gdp.read_file('data/train/train.shp')
train.dropna(subset=['geometry'], inplace = True)
train.reset_index(drop=True, inplace= True)

test = gdp.read_file('data/test/test.shp')
test.dropna(subset=['geometry'], inplace = True)

In [5]:
dates = ["2017-01-01", "2017-01-31", "2017-02-10", "2017-03-12", "2017-03-22", "2017-05-31", "2017-06-20", 
        "2017-07-10", "2017-07-15", "2017-08-04", "2017-08-19"]

In [15]:
def get_nb_polygons_under_n_pixels(df,raster, n):
    # Return the number of polygons that have a width or height under n
    
    nb_under_n = 0
    ## change to the same crs
    df = df.to_crs(raster.crs.data)
    geoms = df.geometry.values
    
    min_height = sys.maxsize
    min_width = sys.maxsize
    max_height = 0
    max_width = 0
    
    
    for index, geom in enumerate(tqdm(geoms)):
        feature = [mapping(geom)] #geojson format

        # the mask function returns an array of the raster pixels within this feature
        out_image, out_transform = mask(raster, feature, crop=True)
        if index == 0:
            print(out_image.shape)
        #print the size of the masked raster
        min_height = min(min_height, out_image.shape[1])
        
        min_width = min(min_width, out_image.shape[2])
        
        max_height = max(max_height, out_image.shape[1])
        
        max_width = max(max_width, out_image.shape[2])

        if(out_image.shape[1] < n or out_image.shape[2] < n):
            nb_under_n +=1
    print('min and max height and width', min_height, min_width, max_height, max_width)
    
    return nb_under_n

In [22]:
# This function is used to mesure how many of the polygons that have either the width or the height less then the size
raster = rasterio.open('mosaics/Mosaic_2017-01-01.tiff')
size = 16
print(get_nb_polygons_under_n_pixels(train, raster, size))
print(get_nb_polygons_under_n_pixels(test, raster, size))

  5%|▌         | 137/2494 [00:00<00:03, 678.73it/s]

(4, 10, 14)


100%|██████████| 2494/2494 [00:03<00:00, 665.87it/s]


min and max height and width 2 3 90 83
1218


 12%|█▏        | 128/1074 [00:00<00:01, 626.49it/s]

(4, 23, 18)


100%|██████████| 1074/1074 [00:01<00:00, 647.06it/s]

min and max height and width 3 1 66 60
517


### crop the raster images and save it to folders

In [23]:
size = 16
parent_path = 'croped_images'

In [25]:
def get_raster_stats(array):
    stats = []
    max_pixel_value = 0
    for band in array:
        max_pixel_value = max(max_pixel_value, band.max())
        stats.append({
            'min': band.min(),
            'mean': band.mean(),
            'median': np.median(band),
            'max': band.max()})
    return (stats, max_pixel_value)

In [72]:
def resize_np_array(arr, size):
    old_shape = arr.shape
    old_size = arr.size
    old_height = old_shape[0]
    old_width = old_shape[1]
    num_channels = old_shape[2]

    
    if(old_height < size and old_width < size): # we don't crop any but we embed it
#         print('case 1')
#         new_im = np.zeros((size,size, num_channels),dtype=np.int)
        new_im = np.zeros((size, size, num_channels))

        x = int((size-old_height)/2)
        y = int((size-old_width)/2)
        new_im[x:x+old_height, y:y+old_width, :] = arr
    
    elif(old_height < size): ## Only the height is smaller
#         print('case 2')            
#         new_im = np.zeros((size,size, num_channels),dtype=np.int)
        new_im = np.zeros((size, size, num_channels))

        x = int((size-old_height)/2)
        y = int((old_width- size)/2)
        new_im[x:x+old_height, :, :] = arr[:, y:y+size, :]
        
    elif(old_width < size): ## Only the height is smaller
#         print('case 3')            
#         new_im = np.zeros((size,size, num_channels),dtype=np.int)
        new_im = np.zeros((size, size, num_channels))

        x = int((old_height- size)/2)
        y = int((size - old_width)/2)
        new_im[:, y:y + old_width, :] = arr[x:x + size, :, :]
        
    else:
#         print('case 4')
        x = int((old_height- size)/2)
        y = int((old_width - size)/2)
        new_im = arr[x:x+size, y:y+size,:]
        
    return new_im

In [160]:
def crop_and_save_train_and_test(date, train_df, test_df, class_col_name, size= 16):
    """
    This function crop polygons from a raster with the size indicated and save it to a folder
    
    date: string:
    The date of the raster image
    
    train_df, test_df: GeoDataFrames:
    The shapefiles containing the polygons in the geometry column
    
    class_col_name: string:
    The column name containing the class of each polygon in the train shapefile
    
    size: int:
    The size of the output images (size * size)
    """
    
    raster = rasterio.open(os.path.join('mosaics', r"Mosaic_"+ date+ ".tiff"), count=4)
    train_df  = train_df.to_crs(raster.crs.data)
    test_df  = test_df.to_crs(raster.crs.data)
    
    ### get the polygons
    geoms_train = train_df.geometry.values
    geoms_test = test_df.geometry.values
            
    for index, geom in enumerate(tqdm(geoms_train)):
        feature = [mapping(geom)] #geojson format
        # the mask function returns an array of the raster pixels within this feature
        out_image, out_transform = mask(raster, feature, crop=True)
        
        class_name = os.path.join(parent_path, date, 'train' + str(size), 'class' + str(train_df[class_col_name].iloc[index]))
        if not os.path.exists(class_name):
            os.mkdir(class_name)
        path_name = os.path.join(class_name , 'index' + str(index)+ '.npy')
                           
        img_full_bands = np.transpose(out_image, (1, 2, 0))            
            
        image_r = img_full_bands[:, :, 2] ## according to sentinel 2 bands specification
        image_r = np.expand_dims(image_r, 2)
        
        image_g = img_full_bands[:, :, 1]
        image_g = np.expand_dims(image_g, 2)
        
        image_b = img_full_bands[:, :, 0]   
        image_b = np.expand_dims(image_b, 2)
        
        image_nir = img_full_bands[:, :, 3]
        image_nir = np.expand_dims(image_nir, 2)
        
        ### Add other layers 
        np.seterr(divide='ignore', invalid='ignore') ## ignore dividing by zero numpy
        ndvi = (image_nir - image_r) / (image_nir + image_r)
        ndvi[np.isnan(ndvi)] = 0
        
        result = np.concatenate([image_r, image_g, image_b, image_nir, ndvi], axis=2)
        resized_result = resize_np_array(result, size)
        np.save(path_name, resized_result)
#         if(index in range(1)):
#             print(result.shape)
#             print(resized_result.shape)
#             print(result[:,:,4])
#             print(resized_result[:, :, 4])
        
    
    for index, geom in enumerate(tqdm(geoms_test)):
        feature = [mapping(geom)] #geojson format
        out_image, out_transform = mask(raster, feature, crop=True)
        
        folder_name = os.path.join(parent_path, date, 'test' + str(size))
        if not os.path.exists(folder_name):
            os.mkdir(folder_name)
        path_name = os.path.join(folder_name, 'index' + str(index)+ '.npy')
                           
        img_full_bands = np.transpose(out_image, (1, 2, 0))            
            
        image_r = img_full_bands[:, :, 2] ## according to sentinel 2 bands specification
        image_r = np.expand_dims(image_r, 2)
        
        image_g = img_full_bands[:, :, 1]
        image_g = np.expand_dims(image_g, 2)
        
        image_b = img_full_bands[:, :, 0]   
        image_b = np.expand_dims(image_b, 2)
        
        image_nir = img_full_bands[:, :, 3]
        image_nir = np.expand_dims(image_nir, 2)
        
        ### Add other layers 
        np.seterr(divide='ignore', invalid='ignore') ## ignore dividing by zero numpy
        ndvi = (image_nir - image_r) / (image_nir + image_r)
        ndvi[np.isnan(ndvi)] = 0
        
        result = np.concatenate([image_r, image_g, image_b, image_nir, ndvi], axis=2)
        resized_result = resize_np_array(result, size)
        np.save(path_name, resized_result)

In [161]:
def print_count_images():
    count = 0
    for dirname, _, filenames in os.walk('./'):
        for filename in filenames:
            if(filename.endswith('npy')):
                count +=1
#                 print(os.path.join(dirname, filename))
    print(count, ' images created')

In [162]:
dates = ["2017-01-01", "2017-01-31", "2017-02-10", "2017-03-12", "2017-03-22", "2017-05-31", "2017-06-20", 
        "2017-07-10", "2017-07-15", "2017-08-04", "2017-08-19"]

In [163]:
for date in dates:
    
    if not os.path.exists(os.path.join(parent_path, date)):
        os.mkdir(os.path.join(parent_path, date))
    if not os.path.exists(os.path.join(parent_path, date, 'train' + str(size))):
        os.mkdir(os.path.join(parent_path, date, 'train' + str(size)))
    if not os.path.exists(os.path.join(parent_path, date, 'test' + str(size))):
        os.mkdir(os.path.join(parent_path, date, 'test' + str(size)))

    crop_and_save_train_and_test(date, train, test, 'Crop_Id_Ne', size)
#     print_count_images()
    



  0%|          | 0/2494 [00:00<?, ?it/s]

  3%|▎         | 66/2494 [00:00<00:03, 659.61it/s]

  5%|▌         | 131/2494 [00:00<00:03, 654.46it/s]

  8%|▊         | 192/2494 [00:00<00:03, 637.37it/s]

 10%|█         | 257/2494 [00:00<00:03, 639.05it/s]

 13%|█▎        | 323/2494 [00:00<00:03, 643.44it/s]

 16%|█▌        | 389/2494 [00:00<00:03, 644.57it/s]

 18%|█▊        | 452/2494 [00:00<00:03, 638.42it/s]

 21%|██        | 515/2494 [00:00<00:03, 633.96it/s]

 23%|██▎       | 579/2494 [00:00<00:03, 635.50it/s]

 26%|██▌       | 644/2494 [00:01<00:02, 639.21it/s]

 28%|██▊       | 709/2494 [00:01<00:02, 642.07it/s]

 31%|███       | 772/2494 [00:01<00:02, 636.77it/s]

 33%|███▎      | 835/2494 [00:01<00:02, 634.09it/s]

 36%|███▌      | 900/2494 [00:01<00:02, 636.14it/s]

 39%|███▊      | 964/2494 [00:01<00:02, 626.22it/s]

 41%|████      | 1028/2494 [00:01<00:02, 630.19it/s]

 44%|████▎     | 1091/2494 [00:01<00:02, 629.10it/s]

 46%|████▋     | 1156/2494 [00:01<00:02, 633.65it/s]



 46%|████▌     | 1150/2494 [00:01<00:02, 559.10it/s]

 48%|████▊     | 1207/2494 [00:02<00:02, 536.55it/s]

 51%|█████     | 1262/2494 [00:02<00:02, 533.67it/s]

 53%|█████▎    | 1316/2494 [00:02<00:02, 534.52it/s]

 55%|█████▍    | 1371/2494 [00:02<00:02, 538.42it/s]

 57%|█████▋    | 1426/2494 [00:02<00:02, 514.61it/s]

 59%|█████▉    | 1478/2494 [00:02<00:02, 487.13it/s]

 61%|██████▏   | 1529/2494 [00:02<00:01, 492.87it/s]

 63%|██████▎   | 1581/2494 [00:02<00:01, 498.41it/s]

 66%|██████▌   | 1635/2494 [00:02<00:01, 509.26it/s]

 68%|██████▊   | 1693/2494 [00:03<00:01, 527.63it/s]

 70%|███████   | 1750/2494 [00:03<00:01, 539.23it/s]

 73%|███████▎  | 1809/2494 [00:03<00:01, 553.36it/s]

 75%|███████▍  | 1865/2494 [00:03<00:01, 541.08it/s]

 77%|███████▋  | 1920/2494 [00:03<00:01, 528.28it/s]

 79%|███████▉  | 1974/2494 [00:03<00:00, 524.77it/s]

 81%|████████▏ | 2031/2494 [00:03<00:00, 535.38it/s]

 84%|████████▍ | 2095/2494 [00:03<00:00, 560.63it/s]

 86%|████████▋ | 2152/2494 [

 61%|██████▏   | 1528/2494 [00:03<00:02, 480.76it/s]

 63%|██████▎   | 1579/2494 [00:03<00:01, 484.24it/s]

 66%|██████▌   | 1634/2494 [00:03<00:01, 501.58it/s]

 68%|██████▊   | 1695/2494 [00:03<00:01, 529.17it/s]

 70%|███████   | 1752/2494 [00:03<00:01, 538.83it/s]

 73%|███████▎  | 1813/2494 [00:04<00:01, 558.17it/s]

 75%|███████▍  | 1870/2494 [00:04<00:01, 535.15it/s]

 77%|███████▋  | 1925/2494 [00:04<00:01, 521.93it/s]

 79%|███████▉  | 1978/2494 [00:04<00:00, 517.83it/s]

 82%|████████▏ | 2041/2494 [00:04<00:00, 547.04it/s]

 84%|████████▍ | 2101/2494 [00:04<00:00, 559.75it/s]

 87%|████████▋ | 2158/2494 [00:04<00:00, 557.78it/s]

 89%|████████▉ | 2215/2494 [00:04<00:00, 556.43it/s]

 91%|█████████▏| 2278/2494 [00:04<00:00, 576.09it/s]

 94%|█████████▎| 2336/2494 [00:05<00:00, 535.50it/s]

 96%|█████████▌| 2393/2494 [00:05<00:00, 543.90it/s]

 98%|█████████▊| 2449/2494 [00:05<00:00, 543.43it/s]

100%|██████████| 2494/2494 [00:05<00:00, 470.70it/s]

  0%|          | 0/1074 [00:

 83%|████████▎ | 2073/2494 [00:03<00:00, 554.29it/s]

 86%|████████▌ | 2136/2494 [00:04<00:00, 572.91it/s]

 88%|████████▊ | 2194/2494 [00:04<00:00, 567.16it/s]

 90%|█████████ | 2254/2494 [00:04<00:00, 574.74it/s]

 93%|█████████▎| 2312/2494 [00:04<00:00, 575.88it/s]

 95%|█████████▌| 2370/2494 [00:04<00:00, 511.75it/s]

 97%|█████████▋| 2423/2494 [00:04<00:00, 514.67it/s]

100%|█████████▉| 2483/2494 [00:04<00:00, 536.26it/s]

100%|██████████| 2494/2494 [00:04<00:00, 526.01it/s]

  0%|          | 0/1074 [00:00<?, ?it/s]

  6%|▌         | 62/1074 [00:00<00:01, 613.44it/s]

 11%|█▏        | 123/1074 [00:00<00:01, 611.18it/s]

 17%|█▋        | 186/1074 [00:00<00:01, 616.21it/s]

 23%|██▎       | 251/1074 [00:00<00:01, 623.68it/s]

 30%|██▉       | 317/1074 [00:00<00:01, 631.75it/s]

 36%|███▌      | 383/1074 [00:00<00:01, 639.59it/s]

 42%|████▏     | 449/1074 [00:00<00:00, 642.98it/s]

 47%|████▋     | 509/1074 [00:00<00:00, 626.53it/s]

 53%|█████▎    | 574/1074 [00:00<00:00, 632.39it/

  6%|▌         | 64/1074 [00:00<00:01, 637.08it/s]

 12%|█▏        | 131/1074 [00:00<00:01, 645.83it/s]

 18%|█▊        | 196/1074 [00:00<00:01, 644.87it/s]

 24%|██▍       | 261/1074 [00:00<00:01, 645.92it/s]

 30%|███       | 327/1074 [00:00<00:01, 649.16it/s]

 37%|███▋      | 396/1074 [00:00<00:01, 659.65it/s]

 43%|████▎     | 464/1074 [00:00<00:00, 664.79it/s]

 49%|████▉     | 530/1074 [00:00<00:00, 663.14it/s]

 56%|█████▌    | 599/1074 [00:00<00:00, 668.43it/s]

 62%|██████▏   | 665/1074 [00:01<00:00, 663.71it/s]

 68%|██████▊   | 730/1074 [00:01<00:00, 638.95it/s]

 74%|███████▍  | 793/1074 [00:01<00:00, 628.01it/s]

 80%|████████  | 860/1074 [00:01<00:00, 639.99it/s]

 86%|████████▌ | 926/1074 [00:01<00:00, 644.49it/s]

 93%|█████████▎| 994/1074 [00:01<00:00, 652.44it/s]

 99%|█████████▉| 1063/1074 [00:01<00:00, 660.72it/s]

100%|██████████| 1074/1074 [00:01<00:00, 652.75it/s]

  0%|          | 0/2494 [00:00<?, ?it/s]

  2%|▏         | 48/2494 [00:00<00:05, 461.33it/s]

  4%

 66%|██████▌   | 711/1074 [00:01<00:00, 650.45it/s]

 73%|███████▎  | 780/1074 [00:01<00:00, 660.97it/s]

 79%|███████▉  | 850/1074 [00:01<00:00, 670.76it/s]

 85%|████████▌ | 918/1074 [00:01<00:00, 672.01it/s]

 92%|█████████▏| 985/1074 [00:01<00:00, 667.62it/s]

 98%|█████████▊| 1052/1074 [00:01<00:00, 660.74it/s]

100%|██████████| 1074/1074 [00:01<00:00, 650.68it/s]

In [150]:
data = np.load('croped_images/2017-01-01/train16/class1/index1026.npy')
data2 = np.load('croped_images/2017-01-31/train16/class1/index1026.npy')
data3 = np.load('croped_images/2017-02-10/train16/class1/index1026.npy')

In [151]:
print(data[:,:,4])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.44861418 0.42835558
  0.41207076 0.35050219 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.47988224 0.4379562  0.43376494
  0.41212744 0.3609831  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.44762846 0.45427944 0.44741358 0.44581343
  0.44416244 0.3786157  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.42836328 0.44126832 0.4419741  0.43522927
  0.43428132 0.34997345 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.46055855 0.41182056 0.44444444 0.44876847 0.44339623
  0.42582988 0.3413929  0.28095601 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.45524915 0.41712378 0.41988024 0.45003647 0.46679688 0.462885

In [152]:
print(data2[:,:,4])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.76538908 0.79836004
  0.77342986 0.67683063 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.77411765 0.80358476 0.79376936
  0.77206312 0.69737123 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.7016507  0.79817198 0.81268546 0.80083288
  0.78727482 0.73924269 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.77334612 0.81022435 0.8044164  0.8070632
  0.78838659 0.72135417 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.75370041 0.79705828 0.8141738  0.81667281 0.8143982
  0.78873239 0.70712909 0.57070451 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.74412855 0.79424944 0.80874722 0.81821507 0.82817366 0.82670401

In [153]:
print(data3[:,:,4])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.73449492 0.77987664
  0.77895772 0.72542226 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.71991701 0.79274411 0.79510529
  0.76860488 0.75127768 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.65841357 0.76937836 0.8074928  0.79943236
  0.78162912 0.7638191  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.73024969 0.79766686 0.79906724 0.80430604
  0.79872513 0.75738397 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.74585171 0.77869822 0.80397453 0.80332887 0.80724749
  0.80358197 0.7507356  0.62788438 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.68082812 0.78945297 0.8003876  0.80841744 0.80968405 0.810805

In [ ]:
size = 8
for date in dates:
    
    if not os.path.exists(os.path.join(parent_path, date)):
        os.mkdir(os.path.join(parent_path, date))
    if not os.path.exists(os.path.join(parent_path, date, 'train' + str(size))):
        os.mkdir(os.path.join(parent_path, date, 'train' + str(size)))
    if not os.path.exists(os.path.join(parent_path, date, 'test' + str(size))):
        os.mkdir(os.path.join(parent_path, date, 'test' + str(size)))

    crop_and_save_train_and_test(date, train, test, 'Crop_Id_Ne', size)
#     print_count_images()
    



  0%|          | 0/2494 [00:00<?, ?it/s]

  3%|▎         | 68/2494 [00:00<00:03, 677.05it/s]

  5%|▌         | 131/2494 [00:00<00:03, 660.27it/s]

  8%|▊         | 195/2494 [00:00<00:03, 651.02it/s]

 10%|█         | 255/2494 [00:00<00:03, 634.16it/s]

 12%|█▏        | 308/2494 [00:00<00:03, 598.34it/s]

 15%|█▍        | 371/2494 [00:00<00:03, 607.23it/s]

 17%|█▋        | 436/2494 [00:00<00:03, 618.58it/s]

 20%|█▉        | 498/2494 [00:00<00:03, 618.35it/s]

 23%|██▎       | 566/2494 [00:00<00:03, 634.97it/s]

 25%|██▌       | 634/2494 [00:01<00:02, 647.29it/s]

 28%|██▊       | 699/2494 [00:01<00:02, 648.04it/s]

 31%|███       | 768/2494 [00:01<00:02, 658.36it/s]

 33%|███▎      | 834/2494 [00:01<00:02, 631.75it/s]

 36%|███▌      | 903/2494 [00:01<00:02, 647.27it/s]

 39%|███▉      | 973/2494 [00:01<00:02, 660.85it/s]

 42%|████▏     | 1043/2494 [00:01<00:02, 669.95it/s]

 45%|████▍     | 1111/2494 [00:01<00:02, 662.85it/s]

 47%|████▋     | 1181/2494 [00:01<00:01, 671.72it/s]



 50%|█████     | 1256/2494 [00:02<00:02, 580.89it/s]

 53%|█████▎    | 1315/2494 [00:02<00:02, 562.46it/s]

 55%|█████▌    | 1373/2494 [00:02<00:01, 567.01it/s]

 57%|█████▋    | 1431/2494 [00:02<00:01, 546.60it/s]

 60%|█████▉    | 1487/2494 [00:02<00:01, 509.08it/s]

 62%|██████▏   | 1546/2494 [00:02<00:01, 529.45it/s]

 64%|██████▍   | 1600/2494 [00:02<00:01, 516.43it/s]

 67%|██████▋   | 1663/2494 [00:03<00:01, 544.45it/s]

 69%|██████▉   | 1726/2494 [00:03<00:01, 565.15it/s]

 72%|███████▏  | 1791/2494 [00:03<00:01, 586.60it/s]

 74%|███████▍  | 1851/2494 [00:03<00:01, 565.97it/s]

 77%|███████▋  | 1909/2494 [00:03<00:01, 553.85it/s]

 79%|███████▉  | 1968/2494 [00:03<00:00, 563.79it/s]

 81%|████████▏ | 2029/2494 [00:03<00:00, 576.41it/s]

 84%|████████▍ | 2094/2494 [00:03<00:00, 596.26it/s]

 87%|████████▋ | 2160/2494 [00:03<00:00, 612.43it/s]

 89%|████████▉ | 2222/2494 [00:03<00:00, 595.24it/s]

 92%|█████████▏| 2289/2494 [00:04<00:00, 615.83it/s]

 94%|█████████▍| 2352/2494 [

 91%|█████████ | 2266/2494 [00:04<00:00, 599.43it/s]

 93%|█████████▎| 2327/2494 [00:04<00:00, 576.31it/s]

 96%|█████████▌| 2386/2494 [00:04<00:00, 580.13it/s]

 98%|█████████▊| 2447/2494 [00:04<00:00, 587.65it/s]

100%|██████████| 2494/2494 [00:04<00:00, 559.50it/s]

  0%|          | 0/1074 [00:00<?, ?it/s]

  7%|▋         | 73/1074 [00:00<00:01, 729.96it/s]

 14%|█▎        | 145/1074 [00:00<00:01, 724.78it/s]

 20%|█▉        | 211/1074 [00:00<00:01, 702.54it/s]

 26%|██▌       | 281/1074 [00:00<00:01, 698.77it/s]

 33%|███▎      | 352/1074 [00:00<00:01, 701.36it/s]

 39%|███▉      | 423/1074 [00:00<00:00, 701.95it/s]

 46%|████▌     | 496/1074 [00:00<00:00, 706.83it/s]

 53%|█████▎    | 569/1074 [00:00<00:00, 711.60it/s]

 59%|█████▉    | 636/1074 [00:00<00:00, 676.01it/s]

 65%|██████▌   | 702/1074 [00:01<00:00, 663.20it/s]

 72%|███████▏  | 771/1074 [00:01<00:00, 670.36it/s]

 78%|███████▊  | 843/1074 [00:01<00:00, 682.17it/s]

 85%|████████▌ | 913/1074 [00:01<00:00, 685.47it/s]



 86%|████████▌ | 924/1074 [00:01<00:00, 716.62it/s]

 93%|█████████▎| 996/1074 [00:01<00:00, 715.93it/s]

 99%|█████████▉| 1068/1074 [00:01<00:00, 711.02it/s]

100%|██████████| 1074/1074 [00:01<00:00, 703.67it/s]

  0%|          | 0/2494 [00:00<?, ?it/s]

  2%|▏         | 51/2494 [00:00<00:04, 503.40it/s]

  4%|▍         | 94/2494 [00:00<00:05, 478.57it/s]

  6%|▌         | 141/2494 [00:00<00:04, 474.94it/s]

  8%|▊         | 194/2494 [00:00<00:04, 488.69it/s]

 10%|█         | 252/2494 [00:00<00:04, 511.31it/s]

 13%|█▎        | 313/2494 [00:00<00:04, 535.52it/s]

 15%|█▍        | 362/2494 [00:00<00:04, 512.70it/s]

 17%|█▋        | 414/2494 [00:00<00:04, 514.10it/s]

 19%|█▊        | 463/2494 [00:00<00:04, 495.73it/s]

 21%|██        | 513/2494 [00:01<00:03, 496.34it/s]

 23%|██▎       | 568/2494 [00:01<00:03, 510.82it/s]

 25%|██▍       | 621/2494 [00:01<00:03, 514.53it/s]

 27%|██▋       | 672/2494 [00:01<00:03, 510.82it/s]

 29%|██▉       | 735/2494 [00:01<00:03, 539.84it/s]

 32%

 27%|██▋       | 668/2494 [00:01<00:03, 508.82it/s]

 29%|██▉       | 726/2494 [00:01<00:03, 526.22it/s]

 32%|███▏      | 793/2494 [00:01<00:03, 560.06it/s]

 34%|███▍      | 852/2494 [00:01<00:02, 568.53it/s]

 37%|███▋      | 919/2494 [00:01<00:02, 593.33it/s]

 39%|███▉      | 982/2494 [00:01<00:02, 603.38it/s]

 42%|████▏     | 1043/2494 [00:01<00:02, 594.65it/s]

 44%|████▍     | 1103/2494 [00:02<00:02, 589.59it/s]

 47%|████▋     | 1163/2494 [00:02<00:02, 554.08it/s]

 49%|████▉     | 1221/2494 [00:02<00:02, 559.66it/s]

 51%|█████     | 1278/2494 [00:02<00:02, 534.06it/s]

 53%|█████▎    | 1333/2494 [00:02<00:02, 537.61it/s]

 56%|█████▌    | 1390/2494 [00:02<00:02, 545.47it/s]

 58%|█████▊    | 1445/2494 [00:02<00:01, 535.71it/s]

 60%|██████    | 1499/2494 [00:02<00:01, 498.41it/s]

 62%|██████▏   | 1558/2494 [00:02<00:01, 522.33it/s]

 65%|██████▍   | 1615/2494 [00:03<00:01, 534.95it/s]

 67%|██████▋   | 1670/2494 [00:03<00:01, 537.89it/s]

 69%|██████▉   | 1733/2494 [00:03<